# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. 

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import RandomState
from scipy import stats as st
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

## Загрузка данных

Произведем выгрузку данных.

In [2]:
try:
    df = pd.read_csv('insurance.csv')

except:
    df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Изучим инфомрацию в представленном датасете.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Из предоставленнно информации видно, что пропущенные значения отсутствуют в датасете. 

Расспотрим более подробно столбец по страховым выплатам. Из него видно, что максисмальное значение страховых выплат для страхуемого составляет 5. Проверим распределение количества выплат.

In [9]:
df[df.duplicated()]

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
281,1,39.0,48100.0,1,0
488,1,24.0,32900.0,1,0
513,0,31.0,37400.0,2,0
718,1,22.0,32600.0,1,0
785,0,20.0,35800.0,0,0
...,...,...,...,...,...
4793,1,24.0,37800.0,0,0
4902,1,35.0,38700.0,1,0
4935,1,19.0,32700.0,0,0
4945,1,21.0,45800.0,0,0


In [6]:
df['Страховые выплаты'].unique()

array([0, 1, 2, 3, 5, 4])

In [7]:
df['Страховые выплаты'].value_counts()

0    4436
1     423
2     115
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64

In [9]:
ratio = df.loc[df['Страховые выплаты'] != 0].count()/ df.loc[df['Страховые выплаты']].count()
ratio

Пол                  0.1128
Возраст              0.1128
Зарплата             0.1128
Члены семьи          0.1128
Страховые выплаты    0.1128
dtype: float64

**Вывод.**

1) Данные предобработаны.
    
2) Процент страховых выплат составляет 11,3%.

## Умножение матриц (Теория. Доказательство корректной работы алгоритма шифрования).

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

По условию задачи нам нужно зашифровать исходные данные и сохранить качество модели. 

Просто умножить  матрицу признаков  на число k не имеет смысла, поскольку 3-ему лицу будет легко догадаться, как можно расшифровать данные.
В качестве шифорвания можно использовать умножение исходной матрицы на случайную обратимую матрицу, которая будет выступать в качестве ключа. Ключ есть только у ограниченного числа пользователей.

Сделаем следующее предположение.

**Предположение:** Пусть $X$ - исходная матрица признаков размера (m,n). Предсказание целевого признака будет производиться по формуле:

$$
a = X w
$$

Возьмем обратимую матрицу $Р$ размера (k,k) из случайных элементов такую, что $k = n$

Тогда предсказания по признакам исходной и зашифрованной матрицы будут равны:
$$
a = a_1
$$

где, $a_1 = X_1 w_1$, $X_1 = XP$


**Обоснование:** Предположим, что предсказания по признакам исходной матрицы $Х$ будут равны предсказаниям по произведению  матрицы $Х$ на матрицу $Р$, то есть $a = a_1$.

На основании данного равенства составим следующее уравнение и будем делать алгебрарические преобразования:

$
a = a_1
$

$
X w = X_1 w_1
$

$
X ((X)^T X)^{-1} (X)^T y = X_1 ((X_1)^T X_1)^{-1} (X_1)^T y
$

Учитывая условия задачи, преобразуем правую чать уравнения.

$
X ((X)^T X)^{-1} (X)^T y = X P ((X P)^T X P)^{-1} (X P)^T y
$

$
X ((X)^T X)^{-1} (X)^T y = X P ((X)^T (P)^T X P)^{-1} (P)^T(X)^T y
$

$
X ((X)^T X)^{-1} (X)^T y = X P (P)^{-1} ((X)^T X )^{-1} ((P)^T)^{-1}  (P)^T (X)^T y
$

$
X ((X)^T X)^{-1} (X)^T y = E X ((X)^T X )^{-1} E (X)^T y
$

Учитывая своство умножения на единичные матрицы $A E = E A = A$ получаем
$X ((X)^T X)^{-1} (X)^T y = X ((X)^T X )^{-1} (X)^T y$ или $a = a_1$.

**Предположение доказано**

## Умножение матриц (Проект)

Для удобства воспользуемся обозначениями, приведенными в теории выше.

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу).

На первом этапе выделим признаки, по котрым будем вести предсказания.

In [10]:
features = df.drop(['Страховые выплаты'], axis = 1)
target = df['Страховые выплаты']

features.shape

(5000, 4)

Чтобы попробовать зашифровать данные, попробуем умножить исходные признаки на обратимую матрицу.

Создадим случайную матрицу и сделаем из нее обратимую, чтобы далее перемножить на признаки.

In [11]:
np.random.seed(4)
P = np.random.normal(3, 2.5, size=(4, 4))

Произведем проверку обратимости матрицы.

Матрица обратима тогда и только тогда, когда она невырождена, то есть её определитель (|Р|) не равен нулю.

In [12]:
np.linalg.det(P)

143.91547866794653

Следовательно матрица является обратимой. Теперь умножим матрицу исходных признаков на обратимую.

In [13]:
X = np.array(features)
Z = X @ P
Z.shape

(5000, 4)

In [14]:
Z

array([[190086.31722309,   6481.55885936, 225572.26968272,
        138082.58633029],
       [145657.71110413,   4949.324121  , 172837.24213868,
        105851.95544556],
       [ 80499.79985182,   2729.59492082,  95520.19993329,
         58511.08296225],
       ...,
       [129905.40054968,   4439.78776184, 154160.08672537,
         94340.79156763],
       [125319.7480295 ,   4288.3752857 , 148707.45731147,
         91022.35671767],
       [155585.28628877,   5312.28186077, 184634.25214955,
        113003.83852479]])

In [15]:
X

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

Для проверки влияния изменения признаков создадим отдельный датасет с признаками Z.

In [16]:
df_Z = pd.DataFrame(data = Z, columns = ['Пол', 'Возраст', 'Зарплата', 'Члены семьи'])
df_Z.head()

,Пол,Возраст,Зарплата,Члены семьи
0,190086.317223,6481.558859,225572.269683,138082.586330
1,145657.711104,4949.324121,172837.242139,105851.955446
2,80499.799852,2729.594921,95520.199933,58511.082962
3,159786.230432,5463.031976,189625.526394,116029.542903
4,100037.159926,3404.479172,118707.367963,72689.487184


Создадим класс LinReg_manual -  логистическая регрессия, написанная вручную.

Для начала проверим на исходных данных.

In [17]:
features_Z = df_Z

class LinReg_manual:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = ((np.linalg.inv(X.T @ X)) @ X.T) @ y# < напишите код здесь >
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinReg_manual()

model.fit(features, target)

predictions = model.predict(features)

r2_score(target, predictions)

0.42494550286668

Проверим на преобразованных данных.

In [18]:
model.fit(features_Z, target)
predictions_Z = model.predict(features_Z)
r2_score(target, predictions_Z)

0.4249455028658199

**Вывод** 

Качество линейной регресси не изменилось. 

Связано это с тем, что умножение матрицы исходных признаков на обратимую матрицу из случайных элементов происходит скалярно и по сути элементы случайной матрицы являются коэфициентами, на которые мы умножаем исходные признаки. Поэтому качество модели не изменяется. 

## Алгоритм преобразования

**Алгоритм**

Учитывая вышеуказанный пункт, порядок алгоритма наиболее подходящего для решения поставленной задачи будет выглядить следующим образом:

1) На вход мы получаем исходный датасет. Назовем его ***df***.

$$
df
$$

2) Выделяем целевой признак и остальные, по которым будем предсказывать.

$$
features
$$

$$
target
$$

3) Создаем матрицу признаков ***X*** для предсказания и проверяем ее размерномть.

$$
X = np.array(features)
$$

$$
(m, n)
$$

4) Создадим квадратную матрицу ***P*** из случайных элементов разменростью (k,k) при условии, что  ширина первой матрицы (𝑚×𝑛) равна высоте второй матрицы В (k×K):

$$
k = n
$$

При этом матрица ***P*** должна быть обратимой (Определитель матрицы Р не должен быть равен 0)

5) Производим матричное умножения ***X*** и ***P***, произведением которых будет является новая матрица ***Z***

$$
Z = X * P
$$

**Обоснование**

Суть шифрования заключается в том, чтобы только определенный круг лиц имел доступ к исходной информации и не происходило утечек. 

В данном случае обратимая матрица будет являться ключем, по которому можно как зашифровать данные (P), так их и расшифровать (P^-1).

При этом, учитывая ранее проведенную проверку и вышесказанное в выводе раздела **Умножение матриц (Проект)** , качество модели линейной регрессии для предсказания количества выплат страховки не будет ухудшаться.

## Проверка алгоритма

Подготовим модель LinReg_2 и включим в нее процесс шифровки исходных признаков.

In [19]:
class LinReg_2:
    def fit(self, train_features, train_target):
        np.random.seed(4)
        P = np.random.normal(3, 2.5, size=(4, 4))
        X = np.array(train_features)
        Z = X @ P
        Z_1 = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = ((np.linalg.inv(Z_1.T @ Z_1)) @ Z_1.T) @ y# < напишите код здесь >
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

Проведем сравнение качества моделей.

In [20]:
model = LinReg_2()

model.fit(features, target)

predictions = model.predict(features)

r2_score(target, predictions)

0.42494550286668

**Вывод.**

Мы видим, что качество модели при добавлении процесса шифрования в класс осталось на прежнем уровне.